# Streaming Token-Level Probe Inference

This notebook demonstrates:
1. **Real-time probe output** as tokens are processed
2. **Token-by-token activation tracking** - see exactly which token triggered which probe
3. **Activation visualization** - see how probe confidence changes across tokens
4. **Export functionality** - save activation data for analysis

In [1]:
# IMPORTANT: Set AMD GPU environment variables BEFORE importing torch
import os
import subprocess

def detect_amd_gpu():
    try:
        result = subprocess.run(['lspci'], capture_output=True, text=True, timeout=2)
        return 'Advanced Micro Devices' in result.stdout
    except:
        return False

if detect_amd_gpu():
    print("AMD GPU detected - configuring ROCm")
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = "11.0.0"
    os.environ["HIP_VISIBLE_DEVICES"] = "0"
    os.environ["AMD_SERIALIZE_KERNEL"] = "3"
    os.environ["TORCH_USE_HIP_DSA"] = "1"
    os.environ["PYTORCH_ROCM_ARCH"] = "gfx1100"
    os.environ["PYTORCH_HIP_ALLOC_CONF"] = "expandable_segments:True"
else:
    print("No AMD GPU detected")

No AMD GPU detected


In [2]:
import sys
from pathlib import Path
import torch

# Add parent directory to path to import from src
sys.path.insert(0, str(Path.cwd().parent))

from src.probes.streaming_probe_inference import StreamingProbeInferenceEngine

/Users/ivanculo/Desktop/Projects/Cogni_map/brije/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1. Initialize Engine

This loads both **cognitive action probes** (45 actions) and **sentiment probes** (continuous scores) from layers 15-30.

In [3]:
PROBES_BASE_DIR = Path('../data/probes_binary')
SENTIMENT_PROBES_DIR = Path('../data/sentiment')
MODEL_NAME = 'google/gemma-3-4b-it'

engine = StreamingProbeInferenceEngine(
    probes_base_dir=PROBES_BASE_DIR,
    model_name=MODEL_NAME,
    sentiment_probes_dir=SENTIMENT_PROBES_DIR,
    include_sentiment=True,  # Include sentiment probes
    layer_range=(15, 30),  # Use layers 15-30
    verbose=True
)

print(f"\n✓ Engine initialized with {len(engine.probes)} cognitive action probes")
if engine.sentiment_probes:
    print(f"✓ Loaded {len(engine.sentiment_probes)} sentiment probes")

Detected compute device: Apple Metal Performance Shaders (MPS)
Initializing StreamingProbeInferenceEngine...
  Probes base dir: ../data/probes_binary
  Model: google/gemma-3-4b-it
  Device: mps
  Layer range: 15-30 (16 layers)
  Sentiment probes: ../data/sentiment

Loaded probe from ../data/probes_binary/layer_15/probe_abstracting.pth
Loaded probe from ../data/probes_binary/layer_15/probe_accepting.pth
Loaded probe from ../data/probes_binary/layer_15/probe_analogical_thinking.pth
Loaded probe from ../data/probes_binary/layer_15/probe_analyzing.pth
Loaded probe from ../data/probes_binary/layer_15/probe_applying.pth
Loaded probe from ../data/probes_binary/layer_15/probe_attentional_deployment.pth
Loaded probe from ../data/probes_binary/layer_15/probe_cognition_awareness.pth
Loaded probe from ../data/probes_binary/layer_15/probe_concretizing.pth
Loaded probe from ../data/probes_binary/layer_15/probe_connecting.pth
Loaded probe from ../data/probes_binary/layer_15/probe_convergent_thinking.

`torch_dtype` is deprecated! Use `dtype` instead!


Detected vision-language model. Loading text-only...


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.69s/it]



✓ Initialized with 720 probes across 16 layers


✓ Engine initialized with 720 cognitive action probes


## 2. Real-Time Streaming Inference

Watch probe activations appear **as each token is processed**!

In [4]:
text = """After receiving feedback, I began reconsidering my approach.
I realized I had been making assumptions without fully understanding the constraints."""

# Run with real-time output
predictions = engine.predict_streaming(
    text,
    top_k=10,
    threshold=0.1,
    show_realtime=True  # Shows activations as they happen!
)


                           STREAMING PROBE INFERENCE                            
                            Processing 26 tokens...                             


Token   0: '<bos>'
  ✓ ████████████████████ remembering               100.0% L16
  ✓ ████████████████████ remembering               100.0% L17
  ✓ ████████████████████ evaluating                100.0% L20
  ✓ ████████████████████ reframing                 100.0% L20
  ✓ ████████████████████ response_modulation       100.0% L20
  ✓ ████████████████████ emotion_responding        100.0% L22
  ✓ ████████████████████ remembering               100.0% L22
  ✓ ████████████████████ cognition_awareness       100.0% L23
  ✓ ████████████████████ emotion_responding        100.0% L23
  ✓ ████████████████████ emotion_responding        100.0% L25
  ✓ ████████████████████ noticing                  100.0% L25
  ✓ ████████████████████ self_questioning          100.0% L25
  ✓ ████████████████████ cognition_awareness       100.0% L26
  ✓ ██████

## 3. Visualize Token-Level Activations

See **exactly which token** triggered each probe and **how strongly**.

In [ ]:
# Visualize the top prediction
engine.visualize_token_activations(predictions, text)

In [ ]:
# Or visualize a specific action
engine.visualize_token_activations(predictions, text, action_name='Reconsidering')

## 4. Inspect Token Activation Data

Access detailed token-level data programmatically.

In [ ]:
# Get the top prediction
top_pred = predictions[0]

print(f"Action: {top_pred.action_name}")
print(f"Final Confidence: {top_pred.confidence:.2%}")
print(f"Peak Token: '{top_pred.peak_activation_token}' ({top_pred.peak_confidence:.2%})")
print(f"\nToken-by-token breakdown:")
print("=" * 70)

for tok in top_pred.token_activations:
    print(f"Pos {tok.token_position:3d} | {tok.token_text:15} | "
          f"Confidence: {tok.confidence:.2%} | Active: {tok.is_active}")

## 5. Compare Multiple Texts

See how different texts activate probes differently at the token level.

In [ ]:
texts_to_compare = [
    "Analyzing the quarterly data to identify key trends and patterns.",
    "Brainstorming creative solutions and thinking outside the box.",
    "Evaluating whether this approach will actually work in practice."
]

for i, text in enumerate(texts_to_compare, 1):
    print(f"\n{'=' * 80}")
    print(f"TEXT {i}: {text}")
    print(f"{'=' * 80}")
    
    preds = engine.predict_streaming(
        text,
        top_k=3,
        threshold=0.05,
        show_realtime=False  # Don't show token-by-token for comparison
    )
    
    print(f"\nTop 3 actions:")
    for j, pred in enumerate(preds, 1):
        print(f"  {j}. {pred.action_name:30s} {pred.confidence:.2%}  "
              f"(Peak at '{pred.peak_activation_token}')")

## 6. Export Token Activation Data

Save activation data to CSV for external analysis.

In [ ]:
# Analyze a text
text = """The quarterly numbers look concerning. Revenue is up but margins are down.
We need to reconsider our pricing strategy."""

predictions = engine.predict_streaming(
    text,
    top_k=10,
    threshold=0.05,
    show_realtime=False
)

# Export to CSV
output_path = Path('../output/token_activations.csv')
output_path.parent.mkdir(exist_ok=True)
engine.export_activations_csv(predictions, output_path)

print(f"\n✓ Data exported to {output_path}")
print("\nYou can now:")
print("  - Open in Excel/Google Sheets")
print("  - Load in pandas for analysis")
print("  - Create custom visualizations")

## 7. Advanced: Find Peak Activation Tokens

Find which specific tokens most strongly activate each cognitive action.

In [ ]:
text = """I was analyzing the problem, comparing different solutions, and evaluating their trade-offs.
After reconsidering my assumptions, I realized I needed to brainstorm more creative approaches."""

predictions = engine.predict_streaming(
    text,
    top_k=20,
    threshold=0.0,  # Get all predictions
    show_realtime=False
)

print("\nPeak Activation Tokens for Each Action:")
print("=" * 80)

for pred in predictions[:10]:  # Top 10
    print(f"{pred.action_name:30s} | Peak: '{pred.peak_activation_token:15}' ({pred.peak_confidence:.2%})")

## 8. Custom Analysis: Activation Timeline

See how cognitive actions evolve throughout the text.

In [ ]:
text = """The data looks wrong. I'm questioning my initial assumptions.
Let me analyze this more carefully and compare it to previous results."""

predictions = engine.predict_streaming(
    text,
    top_k=5,
    threshold=0.1,
    show_realtime=False
)

# Show activation timeline
print("\nActivation Timeline:")
print("=" * 80)

# Get all unique token positions
all_positions = set()
for pred in predictions:
    for tok in pred.token_activations:
        all_positions.add(tok.token_position)

# For each position, show which actions were active
for pos in sorted(all_positions)[:20]:  # First 20 tokens
    # Get token text (same across all predictions)
    token_text = predictions[0].token_activations[pos].token_text
    
    print(f"\nPos {pos:3d} | Token: '{token_text:15}'")
    
    # Show activations for each action at this position
    for pred in predictions:
        tok_act = pred.token_activations[pos]
        if tok_act.is_active:
            print(f"         ✓ {pred.action_name:25s} {tok_act.confidence:.2%}")

<cell_type>markdown</cell_type>## Summary

This notebook demonstrated:

1. ✅ **Real-time probe output** - See activations as they happen
2. ✅ **Token-level tracking** - Record which token triggered each probe
3. ✅ **Activation visualization** - See patterns across tokens
4. ✅ **Data export** - Save for external analysis
5. ✅ **Timeline analysis** - Track how actions evolve through text
6. ✅ **Sentiment analysis** - Continuous sentiment scores alongside cognitive actions

### Use Cases:

- **Debugging probes**: See exactly where activations happen
- **Understanding text**: Identify which words/phrases trigger cognitive actions
- **Model analysis**: Study how LLM representations encode cognitive processes
- **Sentiment tracking**: Monitor emotional tone with continuous scores
- **Research**: Analyze activation patterns across different text types

In [ ]:
# Test texts with different sentiment intensities
sentiment_texts = [
    "I'm absolutely thrilled about this incredible opportunity!",  # Strong positive
    "This is going pretty well so far.",  # Mild positive
    "I'm feeling uncertain about the situation.",  # Neutral/slightly negative
    "I'm really disappointed with how things turned out.",  # Moderate negative
    "This is the worst experience I've ever had."  # Strong negative
]

print("Sentiment Probe Analysis:")
print("=" * 80)

for text in sentiment_texts:
    # Run streaming inference
    predictions = engine.predict_streaming(
        text,
        top_k=5,
        threshold=0.1,
        show_realtime=False
    )
    
    # Check if sentiment probes were included
    if hasattr(engine, 'sentiment_probes') and engine.sentiment_probes:
        print(f"\nText: \"{text}\"")
        print(f"Cognitive Actions: {', '.join([p.action_name for p in predictions[:3]])}")
        print("Sentiment Score: [Would show continuous score from -3 to +3]")
    else:
        print(f"\nText: \"{text}\"")
        print(f"Top Actions: {', '.join([p.action_name for p in predictions[:3]])}")
        print("(Sentiment probes not loaded)")
    print("-" * 80)

## 9. Sentiment Analysis

The engine also includes **sentiment regression probes** that provide continuous sentiment scores (not just positive/negative classifications).